# Warsztaty - Przetwarzanie języka naturalnego

27 lutego 2020, szkolenie Voicelab

Ryszard Tuora

model spaCy dla języka polskiego

składa się z:
- taggera morfosyntaktycznego
- lematyzatora
- parsera zależnościowego
- komponentu NER
- flexera (komponentu do fleksji)

In [0]:
# Przygotowanie środowiska, komendy linux
# instalacja Morfeusza 2
!wget -O - http://download.sgjp.pl/apt/sgjp.gpg.key|sudo apt-key add -
!sudo apt-add-repository http://download.sgjp.pl/apt/ubuntu
!sudo apt update
!sudo apt install morfeusz2
!sudo apt install python3-morfeusz2

# upgrade keras'a
!python3 -m pip install --upgrade keras

# instalacja spaCy

!python3 -m pip install spacy

# instalacja modelu spaCy dla języka polskiego
!wget "http://zil.ipipan.waw.pl/SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz"
!mv 'SpacyPL?action=AttachFile&do=get&target=pl_spacy_model_morfeusz-0.1.0.tar.gz' pl_spacy_model_morfeusz-0.1.0.tar.gz
!python3 -m pip install pl_spacy_model_morfeusz-0.1.0.tar.gz

# linkowanie modelu do spaCy
!python3 -m spacy link pl_spacy_model_morfeusz pl_spacy_model_morfeusz -f

# PO WYKONANIU NALEŻY ZRESETOWAĆ RUNTIME

In [0]:
### PYTHON 3
import keras
print(keras.__version__) # Powinno wyświetlić 2.3.1
# ładowanie modelu
import spacy

nlp = spacy.load("pl_spacy_model_morfeusz")


# Część pierwsza - Tagowanie morfosyntaktyczne
korzystamy z tagsetu NKJP
Nasz tagger to słownikowy tagger Morfeusz2 + dezambiguacja za pomocą neuronowego Toyggera (LSTM)

Każdy token t ma trzy interesujące nas atrybuty: 
- t.tag_ : klasa gramatyczna według polskiego tagsetu NKJP (http://nkjp.pl/poliqarp/help/ense2.html)
- t.pos_ : klasa gramatyczna według międzynarodowego tagsetu UD (mapowana z NKJP)
- t._.feats : customowy atrybut odpowiadający cechom morfosyntaktycznym (np. rodzajowi gramatycznemu, lub liczbie), poszczególne wartości cech są oddzielone dwukropkiem

In [0]:
txt = "Nornica prowadzi zmierzchowo-nocny tryb życia, ale wychodzi również za dnia w poszukiwaniu pokarmu."
doc = nlp(txt) # przetworzenie textu przez pipeline, na wyjściu dostajemy iterowalny obiekty klasy Doc, przechowujący tokeny
for t in doc:
  print(t, t.tag_, t.pos_, t._.feats) # wypisujemy interpretację morfosyntaktyczną każdego tokenu

##Zadanie 1:
Przetwórz tekst znajdujący się w zmiennej txt, oblicz proporcję czasowników (używając tagów UPOS), na podstawie tego oszacuj gatunek do którego należy tekst.

In [0]:
import requests

txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/voicelab_resources/master/1.txt").text

# TODO

doc = nlp(txt)
nouns = 0
verbs = 0
for t in doc:
  if t.pos_ == "NOUN":
    nouns += 1
  if t.pos_ == "VERB":
    verbs += 1
    
print(verbs/nouns)

##Zadanie 2:

Oblicz proporcję rzeczowników w rodzaju żeńskim, do rzeczowników w rodzaju męskim, w tekście pod zmienną txt. Nie czyń założeń co do kolejności cech w t._.feats. Rodzaj męski w tagsecie NKJP jest rozbity na trzy "podrodzaje": m1, m2 i m3.

In [0]:
fem = "lekarka"
print(nlp(fem)[0]._.feats) # sg:nom:f - liczba pojedyncza, mianownik, rodzaj żeński

# TODO

fems = 0
males = 0
for t in doc:
  if t.pos_ == "NOUN":
    if "f" in t._.feats.split(":"):
      fems += 1
    elif not "n" in t._.feats.split(":"):
      males += 1

print(fems/males)


#Część druga - Lematyzacja i własności leksykalne
Nasz model umożliwia słownikową lematyzację przy pomocy Morfeusza, do dezambiguacji (tutaj np. rozróżnienia między "mamy"-> "mama" i "mamy" -> "mieć" służy output taggera).

Lematyzacja pozwala redukować redundancję informacyjną, i ułatwiać zadania takie jak streszczanie, i przeszukiwanie.

Każdy z tokenów dodatkowo jest oznaczony ze względu na pewne własności leksykalne, np. :
- is_stop - słowo należy do stoplisty (listy słów występujących najczęściej, a więc najmniej istotnych semantycznie)
- is_oov - słowo znajduje się poza słownikiem, i.e. embeddingami wykorzystanymi w modelu
- like_url - token ma strukturę url-a
- like_num - token jest liczbą
- is_alpha - token składa się tylko ze znaków alfabetycznych

In [0]:
txt = "Zdenerwowany gen. Leese mówił przez telefon swym podwładnym walczącym pod Monte Cassino, że rozmawia z nimi ze schronu."
doc = nlp(txt)
for t in doc:
  print(t.orth_, t.lemma_, t.is_oov, t.is_stop) # orth_ to atrybut odpowiadający formie słowa występującej w tekście

##Zadanie 3:

Przetwórz tekst spod zmiennej txt, przekonwertuj go do listy lematów, usuń słowa ze stoplisty, oraz interpunkcję, wypisz dziesięć najczęściej pojawiających się lematów. Wypróbuj także opcję w której uwzględniamy tylko rzeczowniki


In [0]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/voicelab_resources/master/2.txt").text

# TODO
doc = nlp(txt)
words = {}
for t in doc:
  if not (t.is_stop or t.tag_ == "interp"):
    try:
      words[t.lemma_] +=1
    except KeyError:
      words[t.lemma_] = 1
listed = [(words[w],w) for w in words]
srt = sorted(listed, reverse = True)
print(srt[0:10])

#Część trzecia - parsowanie zależnościowe

spaCy zawiera parser zależnościowy oparty o metodologię transition-based dependency parsing. 
Interesują nas tu cztery atrybuty:
 - t.head - link do tokenu będącego nadrzędnikiem tokenu t
 - t.dep_ - etykieta opisująca rodzaj zależności
 - t.subtree - generator opisujący poddrzewo rozpięte przez token t
 - t.ancestors - generator opisujący wszystkie przechodnie nadrzędniki tokenu t


In [0]:
txt = "Pierwsza wzmianka o Gdańsku pochodzi ze spisanego po łacinie w 999 Żywotu świętego Wojciecha."
doc = nlp(txt)

import pandas as pd

table = []
for tok in doc:
  tok_dic = {"form": tok.orth_, "label": tok.dep_, "head": t.head.orth_, "subtree": list(tok.subtree), "dep_head": list(tok.ancestors)}
  table.append(tok_dic)
df = pd.DataFrame(table)
print(df.to_string())

### spaCy posiada wbudowaną wizualizację drzew zależnościowych
colab nie pozwala na wyświetlanie etykiet zależności, ale wizualizacje przeprowadzane w innym środowisku będą je już zawierać.

In [0]:
from spacy import displacy

displacy.render(doc, jupyter = True)

###Poniższa funkcja służy łatwej wizualizacji podstawowych własności tokenów z danego tekstu

In [0]:
import pandas as pd

def table(doc):
  table = []
  for tok in doc:
    tok_dic = {"form": tok.orth_, "lemma": tok.lemma_, "tag": ":".join([tok.tag_, tok._.feats]), "dep_label": tok.dep_, "dep_head": tok.head.orth_}
    table.append(tok_dic)
  return pd.DataFrame(table)

txt = "Wiadomość jest symboliczna, ale oznacza też początek długotrwałego trendu.\
 Dochód na mieszkańca z uwzględnieniem realnej mocy nabywczej walut narodowych \
 wyniósł w 2019 r. w Rzeczpospolitej Polskiej 33 891 dolarów, nieco więcej, niż w Portugalii \
 (33 665 dolarów). Jednak Fundusz przewiduje, że w tym roku portugalska gospodarka\
  będzie się rozwijać w tempie 1,6 proc. wobec 3,1 proc. w przypadku gospodarki \
  polskiej. Nożyce między oboma krajami będą się więc rozwierać."

doc = nlp(txt)

tab = table(doc)

print(tab.to_string()) # prosty hack na wypisanie całej tabeli

###Czasami interesują nas większe całości niż pojedyncze tokeny, np. rzeczowniki często łączą się z przymiotnikami w frazy, żeby znajdować takie całości w tekście możemy korzystać z Matchera.

In [0]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)
doc.is_tagged = True # tymczasowy hack, będzie zbędny w kolejnej wersji modelu
pattern = [{"TAG": "adj"}, {"TAG": "subst"}]
matcher.add("AdjSubst", None, pattern) # nazwa, funkcja, wzór

for match_id, start, end in matcher(doc):
  print(doc[start:end])

#### Niemniej w języku polskim, ze względu na swobodę szyku, takie rozwiązanie jest mało owocne, przymiotnik nie musi zawsze poprzedzać rzeczownika który określa, tego typu zależności są widoczne dopiero na poziomie analizy gramatycznej

### W gramatykach zależnościowych możliwa jest rekonstrukcja fraz rzeczownikowych (Noun Phrase - NP)
Aby to zrobić, musimy wydobyć z tekstu wszystkie rzeczowniki, zebrać wszystkie rozpinane przez nie poddrzewa, i wyrzucić ze zbioru te poddrzewa, które są już podfrazą większej frazy rzeczownikowej. To frazy rzeczownikowe, a nie same rzeczowniki, są jednostkami które mają dobrze określone znaczenie. Np. w zdaniu:

###*Żona Pawła jest blondynką.* 

nie mówimy o Pawle, ani o jakiejś abstrakcyjnej żonie, tylko o konkretnej kobiecie, którą wyodrębniamy przez fakt że jest żoną Pawła.

##Zadanie 4:

Napisz funkcję która na wejściu pobiera dokument, a na wyjściu zwraca listę fraz rzeczownikowych zgodnie z podanym powyżej opisem. Następnie wyodrębnij wszystkie frazy rzeczownikowe z tekstu pod zmienną txt.

In [0]:
# TODO

def get_nps(doc):
  # Wyszukuje w sparsowanym dokumencie wszystkie maksymalne frazy rzeczownikowe
  def is_contained(stree, stree_list):
    # Sprawdza czy podana fraza jest podfrazą innej, większej frazy
    return any([all([t in st for t in stree]) for st in stree_list])
  
  nouns = [t for t in doc if t.tag_ == "subst"] # to może być zależne od wersji niemorfeuszowej
  subtrees = [list(n.subtree) for n in nouns]
  to_eliminate = True
  while to_eliminate:
    # pętla do eliminowania fraz będących częścią większych fraz rzeczownikowych
    to_remove = []
    for st in subtrees:
      if is_contained(st, [x for x in subtrees if x != st]):
        to_remove.append(st)
    if to_remove == []:
      to_eliminate = False
    else:
      subtrees = [st for st in subtrees if st not in to_remove]
  return subtrees

for st in get_nps(nlp(txt)):
  print(st)

###Parser zależnościowy pozwala także na dzielenie dokumentów na zdania w sposób bardziej inteligentny, niż posługując się regułami interpunkcji.
###Za zdanie uważamy nieprzerwaną sekwencję tokenów które są powiązane relacjami zależnościowymi.
###Zdania są zapisane w atrybucie doc.sents dokumentu.

In [0]:
for s in doc.sents:
  print(s)

displacy.render(doc, jupyter = True)

##Zadanie 5:

Struktura gramatyczna zdania jest szczególnie istotna psychologicznie. Niektóre własności drzewa zależnościowego mogą sprawiać że zdanie jest trudne do zrozumienia. Do własności tych należy przede wszystkim nieprojektywność, ale także średnia długość zależności (liczona w tokenach .

Napisz funkcję która liczy średnią długość krawędzi zależnościowej w drzewie. Załóżmy że krawędź łącząca słowa sąsiadujące ze sobą ma długość 1. Pamiętaj o tym, ile jest krawędzi w drzewie.

Policz wartość tej statystyki dla podanych zdań.

In [0]:
s0 = "Jan kocha piękną i inteligentną Marię."
s1 = "Lingwistycznie zorientowani filozofowie twierdzą, że wiele problemów dotychczasowej filozofii to pseudoproblemy, wynikające z pojęć pozbawionych precyzyjnej definicji"
s2 = "Zdaniem filozofów paradygmatu lingwistycznego, wiele problemów, którymi dotychczas zajmowała się filozofia było pseudoproblemami, wynikającymi z nieprecyzyjnego zdefiniowania używanych pojęć."
displacy.render(nlp(s1), jupyter = True)


# TODO

def avg_dep_len(doc):
  dlens = 0
  for t in doc:
    if t.dep_ != "ROOT":
      dlens += abs(t.i - t.head.i)
  return dlens / (len(doc) - 1)


print(s0, avg_dep_len(nlp(s0)))
print(s1, avg_dep_len(nlp(s1)))
print(s2, avg_dep_len(nlp(s2)))

#Część czwarta - Rozpoznawanie jednostek nazewniczych (NER)
####Nasz model do spaCy wykorzystuje 6 rodzajów etykiet:
- placeName - miejsca antropogeniczne, np. Dania, Londyn
- geogName - naturalne miejsca geograficzne, np. Tatry, Kreta
- persName - imiona i nazwiska osób, np. J. F. Kennedy, gen. Maczek
- orgName - nazwy organizacji, np. NATO, Unia Europejska
- date - daty, np. 22 marca 1999, druga połowa kwietnia
- time - czas, np. 18:55, pięć po dwunastej

####Nie pozwala na wykrywanie zagnieżdżonych jednostek nazewniczych, np. [placeName: **aleja** [persName: **Piłsudskiego**]]

####Wykryte wzmianki są przechowywane w atrybucie doc.ents dokumentu, każda z tych wzmianek ma atrybut ent.label_, w którym przechowywana jest jej etykieta.

In [0]:
for e in doc.ents:
  print(e, e.label_)

###displaCy pozwala także na wizualizację jednostek nazewniczych

In [0]:
displacy.render(doc, style="ent", jupyter = True)

##Zadanie 6:

W zmiennej txt znajduje się dokument historyczny, wydobądź z niego wszystkie *zdania* zawierające daty, i po kolei je wypisz, rozważ możliwe sposoby automatycznego szeregowania dat wyrażonych w różny sposób (po ewentualnym sprowadzeniu ich do kanonicznej postaci). Rozwiązanie tego problemu jest trudne, i istnieją do niego odrębne narzędzia, umożliwiają one np. automatyczną rekonstrukcję chronologii wydarzeń.

In [0]:
txt = requests.get("https://raw.githubusercontent.com/ryszardtuora/voicelab_resources/master/3.txt").text

# TODO

doc = nlp(txt)
events = []
for s in doc.sents:
  ents = s.ents
  labs = [e.label_ for e in ents]
  if "date" in labs:
    events.append(s)

for s in events:
  print(s)

# Część piąta - podobieństwo w języku
Większość powyższych metod opiera się o embeddingi, czyli funkcje przypisujące słowom wektory w przestrzeni wielowymiarowej (najczęściej 100 lub 300-wymiarowej). Wektory te nie są przypisywane arbitralnie, lecz własności tej przestrzeni mają w jakiś sposób odzwierciedlać własności języka. Np. odległość między wektorami powinna odpowiadać odległości między słowami, którą możemy rozumieć jako podobieństwo znaczeń.

###Bardzo popularny przykład "arytmetyki słów":
znaczenie słów jest reprezentowane przez wektory, dla których mamy zdefiniowane operacje matematyczne. Możemy więc odjąć od znaczenia słowa "królowa", znaczenie słowa "kobieta", i dodać doń znaczenie słowa "mężczyzna", licząc iż wektor będący wynikiem takiego działania odpowiada słowu "król". W praktyce wektor taki najprawdopodobniej nie ma interpretacji, ale możemy znaleźć najbliższy wektor który ma jakąkolwiek interpretację przeszukując słownik.

In [0]:
from scipy import spatial
 
cosine_similarity = lambda x, y: 1 - spatial.distance.cosine(x, y)
 
man = nlp.vocab['mężczyzna'].vector
woman = nlp.vocab['kobieta'].vector
queen = nlp.vocab['królowa'].vector
king = nlp.vocab['król'].vector
 
# We now need to find the closest vector in the vocabulary to the result of "man" - "woman" + "queen"
maybe_king = man - woman + queen
computed_similarities = []
 
for word in nlp.vocab:
    # Ignore words without vectors
    if not word.has_vector:
        continue
 
    similarity = cosine_similarity(maybe_king, word.vector)
    computed_similarities.append((word, similarity))
 
computed_similarities = sorted(computed_similarities, key=lambda item: -item[1])
print([w[0].text for w in computed_similarities[:10]])

###spaCy umożliwia liczenie podobieństwa między słowami 
####jest ono proporcjonalne do cosinusa kąta wektorami je reprezentującymi. Odpowiednia funkcja jest zdefiniowana dla leksemów (tutaj oznaczają one słowa z pominięciem kontekstu).

In [0]:
w1 = "pies"
w2 = "psami"
w3 = "zwierzę"
w4 = "buldog"
w5 = "policjant"
w6 = "strażak"
w7 = "marchewka"
w8 = "słońce"

def similarity(w1, w2):
  w1_lex = nlp.vocab[w1]
  w2_lex = nlp.vocab[w2]
  if w1_lex.has_vector and w2_lex.has_vector:
    sim = w1_lex.similarity(w2_lex)
    print("{} vs. {} -> {}".format(w1, w2, sim))
  else:
    print("Jedno ze słów nie ma reprezentacji wektorowej.")
for w in [w2, w3, w4, w5, w6, w7, w8]:
  similarity(w1, w)

## Zadanie 7:

nlp.vocab może być traktowany jako iterator (stanowi słownik, choć nie wszystkim słowom przypisane są wektory). Napisz funkcję thesaurus(word) która dla podanego słowa znajduje dziesięć najbardziej podobnych słów. Pamiętaj o złożoności obliczeniowej sortowania! 

Możesz wykorzystać lematyzację, oraz tagowanie morfosyntaktyczne, by pozbyć się zbędnych słów.

In [0]:
# TODO
def thesaurus(word):
  synonyms = []
  minimum = 0
  w_lex = nlp.vocab[word]
  for candidate in nlp.vocab:
    if candidate.has_vector and candidate != w_lex:
      similarity = w_lex.similarity(candidate)
      if similarity > minimum:
        synonyms.append((similarity, candidate.orth_))
      if len(synonyms) >10:
        synonyms = sorted(synonyms, reverse = True)
        synonyms = synonyms[:-1]
        minimum = synonyms[-1][0]

  return synonyms

syn = (thesaurus("pies"))
for x in syn:
  print(x)

Nic nie stoi na przeszkodzie by wykorzystywać model do przetwarzania dużych tekstów do których mamy dostęp. 

In [0]:
!wget https://wolnelektury.pl/media/book/txt/homer-iliada.txt
import re

with open("homer-iliada.txt", "r", encoding = "utf-8") as f:
  iliada = f.read()

#with open("ludzie-bezdomni-tom-pierwszy.txt", "r", encoding = "utf-8") as f:
#  iliada = f.read()


# usuwanie podziału na wersy
regex = re.compile(r"([^\n])\n([^\n])")
iliada = regex.sub(lambda m: m[1] + " " + m[2], iliada)
# usuwanie zbędnych pustych linii
regex = re.compile(r"\n{1,}")
iliada = regex.sub("\n", iliada)
#print(iliada)

## dzielenie na paragrafy po tabulatorach i znakach nowej linii
splitregex = re.compile(r"\n| {5}")
paragraphs = [p for p in splitregex.split(iliada) if p != ""]

num_pars = len(paragraphs)
print("Liczba paragrafów: {}".format(num_pars))

docs = []
milestones = [int(num_pars/10) * x for x in range(1,11)]
for i, p in enumerate(paragraphs):
  if i in milestones:
    print("Przetworzono {}% paragrafów".format(int((i / num_pars) * 100)))
  docs.append(nlp(p)) 

Większe jednostki językowe również można reprezentować wektorowo, i liczyć dla nich podobieństwa. Jest to jednak spore uproszczenie, które pomija całkowicie zależności składniowe.

W tym przykładzie próbujemy znaleźć fragment Iliady który pasuje do podanego opisu wydarzeń.

In [0]:
wzorzec1 = nlp("Pryjam płacze nad martwym synem.")
wzorzec2 = nlp("Achilles daje Patroklosowi swój pancerz.")

def find_doc(schema):
  maxsim = 0
  for doc in docs:
    if doc.similarity(wzorzec1) > maxsim:
      maxsim = doc.similarity(wzorzec1)
      maxdoc = doc
  return (maxsim, maxdoc)

print(wzorzec1, find_doc(wzorzec1))
print(wzorzec2, find_doc(wzorzec2))

##Zadanie 8 (dodatkowe):

Flexer to komponent służący do odmiany słów. Użytkownik na wejściu podaje token (i.e. słowo pochodzące z dokumentu, nie sam string!), oraz pożądaną charakterystykę morfosyntaktyczną. Napisz funkcję która wykrywa imię w wiadomości *msg* (nie wybieraj go po pozycji w zdaniu, lecz wykorzystaj NER), i konstruuje wypowiedź postaci "Masz wiadomość od [imię]" gdzie imię zostaje odmienione do dopełniacza.

In [0]:
flexer = nlp.get_pipe("flexer")
# pierwszym argumentem jest token, drugim argumentem jest pożądana charakterystyka morfosyntaktyczna
# można zmienić kilka cech na raz, oddzielając je dwukropkiem, np.
doc = nlp("Napadły na nas psy.")
psy = doc[-2]
flexed = flexer.flex(psy, "gen:sg")
print(flexed)


msg = "Paweł napisał do Ciebie wiadomość."
# TODO
doc = nlp(msg)
blueprint = "Masz wiadomość od {}"
name = [e for e in doc.ents if e.label_ == "persName"][0][0]
gen = flexer.flex(name, "gen")
print(blueprint.format(gen))


'psa'